In [ ]:
import rebound
import time
import sys
import warnings
import numpy as np
from matplotlib import pyplot as plt
from astropy import units as u
from astropy import constants as c

# local imports
import heartbeat
import globs
import metasim
import unhash

In [ ]:
#Yes REBOUND I know I might restore from save...
warnings.filterwarnings("ignore",message="You have to reset function pointers after creating a reb_simulation "
                        "struct with a binary file")
warnings.filterwarnings("ignore",message="File in use for SimulationArchive already exists. "
                        "Snapshots will be appended")

In [ ]:
read_only = True

systems = []

def run_sim(filestem='test/test',tmoons=1e4):
    
    # global variables

    globs.initialise()

    globs.glob_dclo = 1. #CE distance to check in rH
    #dir = 'test/'
    globs.glob_archive = ''
    #globals.glob_names = []
    globs.glob_is_close = False

    system = metasim.MetaSim(filestem=filestem,tmoons=tmoons)
    if not read_only:
        system.run_planets()
        if globs.glob_is_close:
            system.rewind()
            system.add_moons()
            system.run_moons()

    system.analyse()
    system.make_timeline()
    
    systems.append(system)

In [ ]:
N_sys = 100

for i in range(N_sys):
    run_sim(filestem=f'test/test{i:04d}')


In [ ]:
moon_sims = [s for s in systems if s.has_moons]
N_moon_sims = len(moon_sims)
N_surv_hist = [sum([s.npl == i for s in moon_sims]) for i in (0,1,2)]
print(f"# sims where moons added: {N_moon_sims} out of {N_sys}")
print(f"# sims with [0, 1, 2] surviving planets: {N_surv_hist} "
      f"out of {N_moon_sims}")

In [ ]:
where_1_surv = np.where([s.npl == 1 for s in moon_sims])[0]
where_2_surv = np.where([s.npl == 2 for s in moon_sims])[0]
a = [[o.a for o in [s.orb[p] for p in s.surv_pl]] for s in moon_sims]
e = [[o.e for o in [s.orb[p] for p in s.surv_pl]] for s in moon_sims]
n = [[s.n_debris[p] for p in s.surv_pl] for s in moon_sims]

In [ ]:
cmap = 'Set1' 
plt.figure(figsize=(5,4))
plt.scatter([a[w] for w in where_1_surv],[e[w] for w in where_1_surv],c=[n[w] for w in where_1_surv],
            cmap=cmap,marker='*',label='1 planet survives',vmin=0,vmax=8,alpha=0.6,s=100)
plt.scatter([a[w] for w in where_2_surv],[e[w] for w in where_2_surv],c=[n[w] for w in where_2_surv],
            cmap=cmap,marker='o',label='2 planets survive',vmin=0,vmax=8,alpha=0.6)
plt.xscale('log')
plt.xlabel('a [au]')
plt.xlim([0.6,60])
plt.ylim([0,1])
plt.ylabel('e')
plt.legend()
plt.colorbar(label='number bound debris trails')

plt.savefig('a-e.pdf')

In [ ]:
N_debris_1_surv = [n[w] for w in where_1_surv]
N_debris_2_surv = [n[w] for w in where_2_surv]

In [ ]:
N_debris_1_surv_hist = np.array([np.sum(np.array(N_debris_1_surv) == i) for i in (0,1,2,3,4,5,6,7,8)])
N_debris_2_surv_hist = np.array([np.sum(np.array(N_debris_2_surv) == i) for i in (0,1,2,3,4,5,6,7,8)])

In [ ]:
print(f"{np.sum(N_debris_1_surv_hist[1:])} of {np.sum(N_debris_1_surv_hist)} planets in single-planet systems "
      f"have bound debris")
print(f"{np.sum(N_debris_2_surv_hist[1:])} of {np.sum(N_debris_2_surv_hist)} planets in two-planet systems "
      f"have bound debris")

In [ ]:
print(N_debris_1_surv_hist + N_debris_2_surv_hist)
print(N_debris_1_surv_hist)
print(N_debris_2_surv_hist)

In [ ]:
def laplace_rule(r,n):
    
    frac = r/n
    best = (r+1)/(n+2)
    
    theta = np.linspace(0,1,1001)
    
    p = theta**r * (1-theta)**(n-r)
    p = p/np.sum(p)
    
    c = np.cumsum(p)
    
    if r == 0:
        low = 0
        up = np.interp(0.68,c,theta)
    else:
        low = np.interp(0.16,c,theta)
        up = np.interp(0.84,c,theta)
    
    return [frac,best,low,up]

In [ ]:
f_tot = np.zeros(9)
f_1 = np.zeros(9)
f_2 = np.zeros(9)

l_tot = np.zeros(9)
l_1 = np.zeros(9)
l_2 = np.zeros(9)

u_tot = np.zeros(9)
u_1 = np.zeros(9)
u_2 = np.zeros(9)

for i in range(9):
    f,b,l,u = laplace_rule((N_debris_1_surv_hist[i] + N_debris_2_surv_hist[i]),
                           (np.sum(N_debris_1_surv_hist) + np.sum(N_debris_2_surv_hist)))
    f_tot[i] = f
    l_tot[i] = f-l
    u_tot[i] = u-f
    
    f,b,l,u = laplace_rule(N_debris_1_surv_hist[i],np.sum(N_debris_1_surv_hist))
    f_1[i] = f
    l_1[i] = f-l
    u_1[i] = u-f
    
    f,b,l,u = laplace_rule(N_debris_2_surv_hist[i],np.sum(N_debris_2_surv_hist))
    f_2[i] = f
    l_2[i] = f-l
    u_2[i] = u-f

In [ ]:
f_tot = ((N_debris_1_surv_hist + N_debris_2_surv_hist)/
         (np.sum(N_debris_1_surv_hist) + np.sum(N_debris_2_surv_hist)))
f_1 = N_debris_1_surv_hist/np.sum(N_debris_1_surv_hist)
f_2 = N_debris_2_surv_hist/np.sum(N_debris_2_surv_hist)
print(f_tot*100)
print(f_1*100)
print(f_2*100)

In [ ]:
l_tot

In [ ]:
plt.figure(figsize=[5,4])
x = np.arange(0,9)
delta = 0.15
plt.errorbar(x,f_tot,[l_tot,u_tot],fmt='o--',label='All systems')
plt.errorbar(x+delta,f_1,[l_1,u_1],fmt='o--',label='Single-planet')
plt.errorbar(x+2*delta,f_2,[l_2,u_2],fmt='o--',label='Two-planet')
plt.legend()
plt.xlabel('Number of debris trails')
plt.ylabel('Fraction of planets')
plt.savefig('fractions.pdf')

In [ ]:
where_unfinished = np.where([not s.is_stop for s in moon_sims])[0]

In [ ]:
where_unfinished

In [ ]:
[n[w] for w in where_unfinished]

In [ ]:
print(np.mean(N_debris_1_surv))
print(np.mean(N_debris_2_surv))

In [ ]:
i_trail = []
i_pl = []
i_host = []
Om_trail = []
Om_host = []
for s in moon_sims:
    i_pl.extend([np.degrees(p.inc) for p in s.sim.particles if "Planet" in unhash.unhash(p.hash,globs.glob_names)])
    for t in s.trails:
        if t.host in s.surv_pl:
            i_trail.append(np.degrees(t.I))
            i_host.append(np.degrees(s.sim.particles[rebound.hash(t.host)].inc))
            Om_trail.append(np.degrees(t.Om))
            Om_host.append(np.degrees(s.sim.particles[rebound.hash(t.host)].omega))

In [ ]:
plt.figure(figsize=[5,4])

plt.hist(i_trail,density=True,label='Trails')
plt.hist(i_pl,density=True,label='Planets')

plt.legend()

plt.xlabel('Inclination [deg]')
plt.ylabel('Frequency')

plt.show()

In [ ]:
def Imut(I1,O1,I2,O2,rad=False):
    if rad:
        i1,o1,I2,o2 = I1,O1,I2,O2
    else:
        i1 = np.radians(I1)
        o1 = np.radians(O1)
        i2 = np.radians(I2)
        o2 = np.radians(O2)
    
    imut = np.arccos(np.sin(i1)*np.sin(i2)*np.sin(o1)*np.sin(o2)
                    +np.sin(i1)*np.sin(i2)*np.cos(o1)*np.cos(o2)
                    +np.cos(i1)*np.cos(i2))
    
    if rad:
        return imut
    else:
        return np.degrees(imut)

In [ ]:
imut = Imut(i_trail,Om_trail,i_host,Om_host) #I_mut between debris and host planet
imut_t = []
for s in moon_sims:
    for t in s.trails:
        if t.host in s.surv_pl:
            for t2 in s.trails:
                if t != t2 and t.host == t2.host:
                    imut_t.append(Imut(np.degrees(t.I),np.degrees(t.Om),np.degrees(t2.I),np.degrees(t2.Om)))

In [ ]:
# inclinations of disc normal wrt orbital normal for Kenworthy objects
i_los = np.radians(np.array([77.0,77.8,56.8]))
phi = np.radians(np.array([-36.8,18.1,41.2]))

cosI = np.sin(i_los)*np.cos(phi)
I = np.degrees(np.arccos(cosI))
name = ['EPIC 2202','EPIC 2043','V928 Tau']

print(I)

In [ ]:
plt.figure(figsize=[5,4])

plt.hist(imut,density=True,label='Debris vs.\ planet',bins=np.arange(0,180,15),alpha=0.6)
plt.hist(imut_t,density=True,label='Debris vs.\ debris',bins=np.arange(0,180,15),alpha=0.6)
for i in range(len(I)):
    plt.plot([I[i],I[i]],[0,0.012],c='b')
    plt.text(I[i]+2,0.005,name[i],rotation=90)
plt.xlabel('Mutual inclination [deg]')
plt.ylabel('Normalised frequency')
plt.legend()
plt.savefig('inc.pdf')

In [ ]:
# count num planets in a collision
#   0 is moon-moon collision
#   1 is moon-planet TDE
#   2 is planet--planet collision
colltype = np.array([np.sum(["Planet" in n for n in coll.names]) for m in moon_sims for coll in m.colls])

In [ ]:
colltype_hist = [np.sum(colltype == i) for i in range(3)]

In [ ]:
print(colltype_hist)